In [1]:
import lib as lib
import basis_set as bs
import numpy as np
import os
import MC_integration as mc
import matplotlib.pyplot as plt
import scipy.linalg as scp

### Hartree Fock computation of the ground state of the Helium atom

In this Jupyter Notebook we are going to compare the groundstate energies obtained with the analytical formula for the two body integrals with Montecarlo integration. In both cases, the Self-Consistent Field method is used.

First we set up the simulation

In [2]:
N_electrons = 2
N_basis = 4
integrals_file = "integrals_He.npy"

normalized_wf = False

max_iter_SCF = 500
eps_SCF = 1E-5
Delta_SCF = 0

In [3]:
def integrand_2(R, indices):
    r1 = R[:,:,0:3]
    r2 = R[:,:,3:6]
    r12 = np.sqrt(np.sum((r1 - r2)**2, axis=-1))
    return 1/r12

Now we make minor modifications on the integral_master class to adapt it to the computation of the Helium atom, whose basis functions are different.

In [4]:
class integral_master_He():
    """
    Calculates, stores and retrieves the values of the <pr|g|qs> integrals
    """
    def __init__(self, dimension):
        """
        Initialization of the object

        Parameters
        ----------
        dimension : int
            Number of single basis functions

        Returns
        -------
        None
        """

        self.integral_dict_1 = None
        self.integral_dict_2 = None
        self.dimension = dimension

        return

    def calculate(self, file_name):
        """
        Calculates the <pr|g|qs> integrals and stores them in file

        Parameters
        ----------
        file_name : str
            File name that stores the values of the integrals

        Returns
        -------
        None
        """

        if file_name in os.listdir():
            print("Integral file already exists. Not computing the integrals. ")
            self.load_integrals(file_name)
            return

        integral_dict_1 = {}
        integral_dict_2 = {}

        # 1-body integrals
        for p in range(1, self.dimension + 1):
            for q in range(1, self.dimension + 1):
                I = self.calculate_1(p, q)

                integral_dict_1[(p, q)] = I

        # 2-body integrals
        for p in range(1, self.dimension + 1):
            for q in range(1, p + 1):
                for r in range(1, p):
                    for s in range(1, r + 1):
                        I = self.calculate_2(p, r, q, s)

                        integral_dict_2[(p, r, q, s)] = I
                        integral_dict_2[(q, r, p, s)] = I
                        integral_dict_2[(p, s, q, r)] = I
                        integral_dict_2[(r, p, s, q)] = I
                        integral_dict_2[(q, s, p, r)] = I
                        integral_dict_2[(r, q, s, p)] = I
                        integral_dict_2[(s, p, r, q)] = I
                        integral_dict_2[(s, q, r, p)] = I
                r = p
                for s in range(1, q + 1):
                    I = self.calculate_2(p, r, q, s)

                    integral_dict_2[(p, r, q, s)] = I
                    integral_dict_2[(q, r, p, s)] = I
                    integral_dict_2[(p, s, q, r)] = I
                    integral_dict_2[(r, p, s, q)] = I
                    integral_dict_2[(q, s, p, r)] = I
                    integral_dict_2[(r, q, s, p)] = I
                    integral_dict_2[(s, p, r, q)] = I
                    integral_dict_2[(s, q, r, p)] = I

        np.save(file_name, np.array([integral_dict_1, integral_dict_2]))

        self.load_integrals(file_name)

        return


    def load_integrals(self, file_name):
        """
        Loads the values of the integrals in this object

        Parameters
        ----------
        file_name : str
            File name that stores the values of the integrals

        Returns
        -------
        None
        """

        self.integral_dict_1, self.integral_dict_2 = np.load(file_name, allow_pickle = True)

        return

    def get_1(self, p, q):
        """
        Returns the value of the h_pq integrals from the class dictionaries

        Parameters
        ----------
        p, q: int
            Indices that specify the h_pq integral

        Returns
        -------
        I : float
            Value of the h_pq integral
        """

        I = self.integral_dict_1[(p, q)]

        return I

    def get_2(self, p, r, q, s):
        """
        Returns the value of the <pr|g|qs> integrals from the class dictionaries

        Parameters
        ----------
        p, r, q, s: int
            Indeces that specify the <pr|g|qs> integral

        Returns
        -------
        I : float
            Value of the <pr|g|qs> integral
        """

        I = self.integral_dict_2[(p, r, q, s)]

        return I


    
    def calculate_1(self, p, q):
        """
        Calculates the value of the h_pq integrals by direct integration

        Parameters
        ----------
        p, q: int
            Indices that specify the h_pq integral

        Returns
        ----------
        I : float
            Value of the h_pq integral
        """

        I = 3*bs.ALPHA[p-1]*bs.ALPHA[q-1]*np.pi**(1.5)/(bs.ALPHA[p-1] + bs.ALPHA[q-1])**(5/2) - 4*np.pi/(bs.ALPHA[p-1] + bs.ALPHA[q-1])

        return I

    def calculate_2(self, p, r, q, s):
        """
        Calculates the value of the <pr|g|qs> integrals by monte carlo integration methods

        Parameters
        ----------
        p, q, r, s: int
            Indices that specify the <pr|g|qs> integral

        Returns
        ----------
        I : float
            Value of the <pr|g|qs> integral
        """

        system_size = 5
        N_walkers = 500
        N_steps = 12000
        N_skip = 2000
        trial_move = None
        
        integrand = integrand_2
        indices = np.array([p,r,q,s])
        dimension = 6
   
        cov = 0.5*np.diag([1/(bs.ALPHA[p-1] + bs.ALPHA[q-1])]*3 + [1/(bs.ALPHA[r-1] + bs.ALPHA[s-1])]*3)
        steps = np.random.multivariate_normal(np.zeros(6), cov, N_walkers*N_steps) # shape = N_walkers*N_steps, 6
        steps = steps.reshape(N_walkers, N_steps, 6)

        E_local = integrand_2(steps, indices)
        E_walkers = np.average(E_local, axis=0)
        I_2 = np.average(E_walkers)
        deltaI_2 = np.std(E_walkers)/np.sqrt(N_walkers)
        
        a = bs.ALPHA[p-1] + bs.ALPHA[q-1]
        b = bs.ALPHA[r-1] + bs.ALPHA[s-1]
        PROD = (np.pi/a)**1.5 * (np.pi/b)**1.5
        
        #I_anal = 2*np.pi**2.5 / ((bs.ALPHA[p-1] + bs.ALPHA[q-1])*(bs.ALPHA[r-1] + bs.ALPHA[s-1])*np.sqrt((bs.ALPHA[p-1] + bs.ALPHA[q-1] + bs.ALPHA[r-1] + bs.ALPHA[s-1])))

        
        return I_2*PROD
        #return I_anal


### Computation block
In the next piece of code we run the actual computation of the energy levels of the Helium atom. First we initialize the class to compute the integrals using the four basis functions defined in Jos' book and then we initialize a zero coeficient matrix and we compute the density matrix. From there we actually compute the matrix and then we check whether the wave functions are normalized and compute the matrix S accordingly.

In [5]:
integrals = integral_master_He(N_basis)
C = np.zeros((N_basis, N_basis))
rho = lib.density_matrix(C, N_electrons)
print('C',C)
print('rho', rho)
    # One- and Two-body integrals
integrals.calculate(integrals_file)

    # Normalization of wave functions
if not normalized_wf:
    S = np.zeros((4,4))
    for p in range(4):
        for q in range(4):
            S[p][q] = (np.pi/(bs.ALPHA[p]+bs.ALPHA[q]))**(1.5)
    SVAL, SVEC = np.linalg.eigh(S) 
    SVAL_minhalf = (np.diag(SVAL**(-0.5))) 
    X = np.dot(SVEC, np.dot(SVAL_minhalf, np.transpose(SVEC)))
else:
    S = np.eye(N_basis)
print('S',S)

C [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
rho [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
S [[1.20975063e+01 2.91187719e+00 3.71330014e-01 2.30637530e-02]
 [2.91187719e+00 1.42134692e+00 2.99025043e-01 2.22459699e-02]
 [3.71330014e-01 2.99025043e-01 1.41564997e-01 1.89120383e-02]
 [2.30637530e-02 2.22459699e-02 1.89120383e-02 8.24921040e-03]]


Finally, we run the self consistent field iteration to recursively update C until a convergence condition is fulfilled. At every step, we calculate a new density matrix according to the newly obtained eigenstates. However, this usually leads to oscillations and the algorithm does not converge. Therefore, at every step we do an average between the old and new density matrices according to some randomly chosen coefficient $0<\alpha <1$, that is,

$\rho_{\text{avg}} = \alpha \rho_{\text{new}}+(1-\alpha)\rho_{\text{old}}.$

In this way, the SCF iterations usually converge. Sometime it might not, which is why there is a `while` loop to keep trying until it converges.

In [8]:
# Self Consistent Field
converged = False
while not converged:
    n_iterations = 0
    rho_old = np.zeros((N_basis, N_basis))
    C = np.zeros((N_basis, N_basis))
    rho = lib.density_matrix(C, N_electrons)

    while n_iterations < max_iter_SCF:
        n_iterations += 1

        F = lib.create_F_matrix(rho, integrals)

        if normalized_wf:
            E, C = np.linalg.eigh(F)
        #else:
            #F_prime = np.conjugate(X.transpose()) @ F @ X
            #E, C_prime = np.linalg.eigh(F_prime)
            #C = X @ C_prime
        else:
            E, C = scp.eigh(F, S)

        rho = lib.density_matrix(C, N_electrons)

        if lib.delta_rho(rho, rho_old) > Delta_SCF:
            alfa = np.random.rand()
            rho = alfa*rho+(1-alfa)*rho_old

        total_E = lib.total_energy(rho, F, integrals)

        if lib.delta_rho(rho, rho_old) < eps_SCF:
            converged = True
            break

        total_E_old = total_E
        rho_old = rho

    print("E = {:0.7f} | N(SCF) = {}".format(total_E, n_iterations))

E = -2.8550787 | N(SCF) = 166


Using the analytical expressions for the two-body integrals from Jos' book, we obtain the following groundstate energy

In [9]:
N_electrons = 2
N_basis = 4
integrals_file = "integrals_He_anal.npy"

normalized_wf = False

max_iter_SCF = 500
eps_SCF = 1E-5
Delta_SCF = 0

In [10]:
integrals = integral_master_He(N_basis)
C = np.zeros((N_basis, N_basis))
rho = lib.density_matrix(C, N_electrons)
print('C',C)
print('rho', rho)
    # One- and Two-body integrals
integrals.calculate(integrals_file)

    # Normalization of wave functions
if not normalized_wf:
    S = np.zeros((4,4))
    for p in range(4):
        for q in range(4):
            S[p][q] = (np.pi/(bs.ALPHA[p]+bs.ALPHA[q]))**(1.5)
    SVAL, SVEC = np.linalg.eigh(S) 
    SVAL_minhalf = (np.diag(SVAL**(-0.5))) 
    X = np.dot(SVEC, np.dot(SVAL_minhalf, np.transpose(SVEC)))
else:
    S = np.eye(N_basis)
print('S',S)

C [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
rho [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Integral file already exists. Not computing the integrals. 
S [[1.20975063e+01 2.91187719e+00 3.71330014e-01 2.30637530e-02]
 [2.91187719e+00 1.42134692e+00 2.99025043e-01 2.22459699e-02]
 [3.71330014e-01 2.99025043e-01 1.41564997e-01 1.89120383e-02]
 [2.30637530e-02 2.22459699e-02 1.89120383e-02 8.24921040e-03]]


In [11]:
# Self Consistent Field
converged = False
while not converged:
    n_iterations = 0
    rho_old = np.zeros((N_basis, N_basis))
    C = np.zeros((N_basis, N_basis))
    rho = lib.density_matrix(C, N_electrons)

    while n_iterations < max_iter_SCF:
        n_iterations += 1

        F = lib.create_F_matrix(rho, integrals)

        if normalized_wf:
            E, C = np.linalg.eigh(F)
        #else:
            #F_prime = np.conjugate(X.transpose()) @ F @ X
            #E, C_prime = np.linalg.eigh(F_prime)
            #C = X @ C_prime
        else:
            E, C = scp.eigh(F, S)

        rho = lib.density_matrix(C, N_electrons)

        if lib.delta_rho(rho, rho_old) > Delta_SCF:
            alfa = np.random.rand()
            rho = alfa*rho+(1-alfa)*rho_old

        total_E = lib.total_energy(rho, F, integrals)

        if lib.delta_rho(rho, rho_old) < eps_SCF:
            converged = True
            break

        total_E_old = total_E
        rho_old = rho

    print("E = {:0.7f} | N(SCF) = {}".format(total_E, n_iterations))

E = -2.8551625 | N(SCF) = 66


So we see that the groundstate energies are

E = -2.8550787 | N(SCF) = 166, with Montecarlo Integration for two-body integrals

E = -2.8551625 | N(SCF) = 66, with analytical two-body integrals